In [ ]:
from IPython.display import Image, HTML
%matplotlib inline

# Blind Deconvolution
**Authors**: [Scott Sievert](http://scottsievert.com), Gunnet Singh Mehta, Smita Bagewadi

## Deconvolution
[Deconvolution] is a classic problem in signal processing. Given observations $y$ from an LTI system or $$y = h \otimes x + n$$ with *exact* knowledge of $h$, the true signal $x$ would like to be estimated.

Many methods exist to solve this problem. Any of the methods used in Lab 4 would apply, but there are more methods. We could use any of the following to get a good estimate of $x$ from knowledge of $y$ and $h$:

* [Wiener filter]
* [TV denoising] (see [Beck et. al](#References))
* $\ell_2$ regularization (aka [Tikhonov regularization]) to ensure that our estimate doesn't grow large as our noise increases
* $\ell_1$ regularization, using sparsity in the wavelet basis
* [Richardson-Lucy deconvolution], a method using Baye's Theorem

The methods can be remarkably well suited when $h$ is *exactly* known. For example, when we use $\ell_2$ regularization and form an estimate $\widehat{x}$ with
$$\widehat{x} = \arg \min_x \left|\left|y - h\otimes x\right|\right|_2^2 + \lambda \cdot||x||_2^2$$

The below widget shows the implementation of this approximation. Note that this widget is remarkably noise tolerant because the regularization parameter is enforcing that all values must remain close to 0.

(note: you can play with this! You can change the observed noise and the regularization parameter. This is much more noise tolerant than the inverse filter, which gives *exact* reconstruction when no noise added but the second noise is added the values increase unrealistically)

[Richardson-Lucy deconvolution]:https://en.wikipedia.org/wiki/Richardson–Lucy_deconvolution
[Tikhonov regularization]:https://en.wikipedia.org/wiki/Tikhonov_regularization
[TV denoising]:https://en.wikipedia.org/wiki/Total_variation_denoising
[Wiener filter]:https://en.wikipedia.org/wiki/Wiener_filter
[Deconvolution]:https://en.wikipedia.org/wiki/Deconvolution

In [ ]:
HTML('figures/widgets/2015-inverse-1-blur_l2_reg.html')

## Blind Deconvolution
[Blind deconvolution] is a much more real-world example of deconvolution. We might not *exactly* know the filter. We might know it's a Gaussian filter function but don't exactly know how wide it is -- we might have an estimate for $\sigma$ but not know it exactly.

[Blind deconvolution]:https://en.wikipedia.org/wiki/Blind_deconvolution

Mathematically, we are given possibly noisy observations $y$ and asked to estimate $x$ without knowledge of $h$.
$$y = h \otimes x + n$$
This is an extremely ill-posed problem. We only know the structure for $y$ but we don't know $x$, $h$ or $n$. In general, this problem is impossible to solve -- we only have one known variable $y$ and three unknown variables $x$, $h$ and $n$. We might not even know the size of $h$!

It becomes tractable once we have an *estimate* for $h$. In practice, this often means that we know $h$ is a Gaussian blur but are unsure of the blur variance.

## Applications
The [Hubble telescope flawed mirror] is by far the best example of blind deconvolution. When it launched in 1992, the mirror had been flawed on launch. Later analysis showed that it was only warped by a small amount (roughly 10 nanometers). This error happened during manufacturing, even though it was the most precisely designed mirror to date.

[Hubble telescope flawed mirror]:https://en.wikipedia.org/wiki/Hubble_Space_Telescope#Flawed_mirror

This means that when the snapped the first picture, they got the image on the left when they were expecting the image on the right:

In [ ]:
Image('figures/hubble.jpg')

This sparked off research in blind deconvolution. The camera that snapped this photo existed in space meaning it would have been expensive to fix (although they eventually did fix it with more mirrors as described in [Design of a solution]).

[Design of a solution]:https://en.wikipedia.org/wiki/Hubble_Space_Telescope#Design_of_a_solution

## Theoretical approach
We observe $y = h \otimes x + n$ and it can be assumed we have an estimate for $h$ as $\widehat{h}$. We observe $y$ and would like to estimate $x$ in the presence of noise.

To solve this problem, we will use the alternating minimization algorithm, which is related to [biconvexity]. We will fix $\widehat{h}$ and find the optimal estimate of our original signal, $\widehat{x}$. We will then fix $\widehat{x}$ and optimize for $\widehat{h}$. Pictorially,

[biconvexity]:https://en.wikipedia.org/wiki/Biconvex_optimization

In [ ]:
Image('figures/loop.png', width=400)

Formally, we can represent this by saying that our current estimate at iteration $k$ is

$$\begin{align}
x_{k} &= f\left(y, h_{k-1}\right)\\
h_{k} &= f\left(y, x_{k}\right)
\end{align}$$

For more detail on this alternating minimization method, see [Gorski et. al.]. This method says “fix one variable and optimize for the other variable from you observations” and go back and forth. This method is widespread; we found it in every blind deconvolution reference we found, and alternating minimization algorithms are even found in other areas such as [biconvex optimization] (through the [proximal gradient method]) and unsupervised learning (at least for [dictionary learning]).

[dictionary learning]:https://en.wikipedia.org/wiki/Machine_learning#Sparse_dictionary_learning

These steps can be summarized in two steps:

1. Take a step in the direction that optimizes $X_{k+1}$ with knowledge of $Y$ and $H_k$.
2. With this new estimate $X_{k+1}$ and knowledge of $Y$, optimize for $H_{k+1}$.

We can treat these methods like black boxes and just look at the input and output. We are finding a better estimate; how we produce it doesn’t affect practical performance (although it may be harder to analyze from a theoretical standpoint).

[proximal gradient method]:https://en.wikipedia.org/wiki/Proximal_gradient_methods_for_learning
[biconvex optimization]:https://en.wikipedia.org/wiki/Biconvex_optimization
[Gorski et. al.]:http://www2.math.uni-wuppertal.de/~klamroth/publications/gopfkl07.pdf

Below, we will describe Richardson-Lucy blind deconvolution. However, this is not the only approach to this problem and other methods exist. Some of these methods are

* Total variation blind deconvolution ([Chan et al][ref])
* Blind deconvolution using a normalized sparsity measure ([Krishan et. al][ref])

[ref]:#References



### Richardson-Lucy deconvolution
The Richardson Lucy deconvolution algorithm is the basis for the blind deconvolution algorithm. The algorithm was developed in the early 1970s by Richardson and Lucy. It was rederived by Shepp and Vardi as an algorithm to solve positron imaging problems. This algorithm has become particularly famous in astronomy and medical imaging where there is high level of noise and the imaging equipment characteristics often changes due to factors beyond control. In simple terms, the impulse filter function h can vary over time.

The algorithm is based on Bayes’s Theorem and can be found in [Fish et. al.](#References)

$$P(x|y) = \frac{P(y|x) \cdot P(x)}{\int P(y|x) P(x) dx}$$

* $P(Y|X)$ - is the conditional probability of occurrence of y given x
* $P(X)$ - probability of occurrence of X
* $P(X|Y)$ - inverse conditional probability of the x being a particular value given Y was the input

Intuitively $P(Y|X)$ is the rule of value of Y given X is an input, which in case of image is the actual blurring kernel. $P(X)$ is an equivalent of the actual image i.e. the input to the system. $P(X|Y)$ is the estimate for the input image. This Bayesian rule can be converted into image domain variables to

$$x_{i+1}(l) = \left[\left( \frac{y(l)}{x_i(l) \otimes h(l)}\right) \otimes h(-l) \right] x_i(l)$$

where $l$ is some location. Note that in Richardson Lucy algorithm we assume that we know the kernel exactly. Here $h(-l)$ is the mirror image of function $h(l)$. This is in one dimension, but it is easily generalizable to two dimensions.




### Richardson Lucy Algorithm in blind deconvolution
The Richardson-Lucy Algorithm is agnostic to the convolution operator: it makes no assumptions about whether the $f$ or $g$ is the convolution operator. Given the alternating minimization approach described in [Theoretical approach](#Theoretical-approach), that means the blind deconvolution approach is simple:

1. Use the Richardson-Lucy algorithm to estimate $g$ from $c$ and $f$
2. Use the Richardson-Lucy algorithm to estimate $f$ from $c$ and $g$.
3. Loop back to (1) until convergence criterion satisfied.

In mathematics, that takes the following form:

$$\begin{align}
x_{i+1}^k(l) &= \left[\left( \frac{y(l)}{x^k_i(l) \otimes h^{k-1}(l)}\right) \otimes h^k(-l) \right] x^k_i(l)\\
h_{i+1}^k(l) &= \left[\left( \frac{y(l)}{h^k_i(l) \otimes x^{k}(l)}\right) \otimes x^{k-1}(-l) \right] h^k_i(l)\\
\end{align}$$

The above equations are the Richardson Lucy implementation for the blind deconvolution problem. In the equation for $h^{k}_{i+1}$ we find a better estimate of the filter by using the observed output $y$, the estimate for the input image $x^k$ and the *current* estimate for the filter or $h_k$. The estimation for the filter is refined iteratively.

A similar process is repeated to get a better estimate for the image $x_{i+1}^k$ using the refined filter value. It does this based on the Bayesian theory for what $x_{i+1}^k$ *should* given the current outputs. Intuitively, it may not be very clear to a reader what refinement is being done to the image and filter at each step.

It was observed in implementation that this algorithm did not perform well when the refinement for image estimate $x$ and $h$ immeditely after one another (i.e., only 1 refinement done to estimate $x$ and $h$). The algorithm performed better when both filter and image were refined multiple times (7 times) before moving on to refine image/kernel.

The method of reconstruction of image from an estimate of kernel could have been replaced by any one of the methods (Weiner, Tikhinov regularisation, Richardson Lucy deconvolution). In practice we used Weiner filtering because it required less tuning of parameters for different images.

## Implementation
Given this setup, the implementation is relatively straightforward. Given possibly noisy observations $y$ and an *estimate* for the convolution kernel $h_0$, we can step into a for-loop to predict each variable. [The complete implementation] can be found on GitHub, but we also describe it below.

In the implementation below, we say that our estimate for $h$ is a Gaussian blurring function, and the true $h$ is a Gaussian blurring function; this is a reasonable estimate. However, we get the width of the Gaussian blurring function wrong by a factor of 1.5.

[The complete implementation]:https://github.com/scottsievert/ece533_finalproj

In [ ]:
clc; clear all; close all;
f = functions_();

% can see results with blur_sigma=6, iterations=8, lucy_iterations=7, sigma=1e-4
blur_sigma = 6;
[x, X, h, H, y, Y, n] = f.input_and_observations(blur_sigma);
blur_sigma_hat = 1.5*blur_sigma;
[~, ~, h_hat, ~, ~, ~, ~] = f.input_and_observations(blur_sigma_hat);

% How many iterations are required to get a good approximation for the variable
% we're optimizing before?
lucy_iterations = 7;

% How many times do we switch back and forth between optimizing for each
% variable?
iterations = 20;
for k=1:iterations,
    % Optimize for x_hat. These values can be treated as a black box; we can
    % swap any of the out (makes sense by looking at the input/outputs)
    x_hat = deconvlucy(y, h_hat, lucy_iterations);
    %x_hat = deconvwnr(y, h_hat);
    %x_hat = deconvreg(y, h_hat);

    % Estimate H. Richardson-Lucy doesn't make any assumptions about the form of
    % the equation and X and H are interchangable. It goes of the Fourier
    % tranfrom representation.
    h_hat = deconvlucy(y, x_hat, lucy_iterations);
end

% display the results (can be found on GitHub)

In this implementation, we use Matlab’s `deconvlucy`. We ran into sizing issues while trying to estimate $h$ from $x$ and $y$. We tried getting around this by representing the Richardson-Lucy deconvolution in the Fourier domain, but ran into resizing issues combined with implementation issues (meaning we didn’t know when we had a correct solution to the resizing issues). In the end, we decided to go with the Matlab implementation of Richardson-Lucy deconvolution, `deconvlucy`. [The Matlab implementation of `deconvlucy`] performs more checks and is more than other Richardson-Lucy implementations such as [scikit-image’s `richardson-lucy`]. The Matlab implementation of `deconvlucy` shows that it goes to extra effort to avoid size issues.

[scikit-image’s `richardson-lucy`]:https://github.com/scikit-image/scikit-image/blob/master/skimage/restoration/deconvolution.py#L333
[The complete implementation]:https://github.com/scottsievert/ece533_finalproj
[The Matlab implementation of `deconvlucy`]:http://read.pudn.com/downloads112/sourcecode/graph/texture_mapping/464953/images/deconvlucy.m__.htm

## Results
Blind deconvolution is very sensitive to initial estimates and parameters because this is an ill-posed problem. To simulate a real-world process, we knew the form of the transfer function (a Gaussian) but not the variance/width. This would simulate a situation where we know our image is blurred by our camera/telescope but are uncertain exactly how it was blurred. To simulate this, our initial estimate is a Gaussian with a standard deviation of 1.5 times the true standard deviation. Using this initial estimate, we have run the deconvolution process and have plotted the results.

Then running the deconvolution process as mentioned in [*Implementation*](#implementation), we obtain the following results.


In [ ]:
Image(filename='figures/lucy_lucy.png')

In this figure, the top row shows the ground truth image and the initial estimate for the convolution kernel. The second row shows our observations of the image and our estimate after the entire blind deconvolution process. We have also shown the true kernel and estimated kernel in the last row.

When thinking of the blind deconvolution process in terms of black boxes, we can replace the Richardson-Lucy deconvolution to estimate $x$ with knowledge of $y$ and $h_k$ with any similar deconvolution process. This could be inverse filtering, wavelet sparsity regularization, TV denoising, Wiener filtering or any other deconvolution algorithm.

After replacing this “estimate $x$ from $y$ and $h_k$” Richardson-Lucy block with an equivalent Wiener filter block, we get the following image:

In [ ]:
Image(filename='figures/lucy_weiner.png')

(this image follows the same format as the figure above)

This gives sharper results, but is shifted. We might have prior knowledge that this blurring function’s center was at the origin (i.e., the Hubble telescope mirror had only been warped but not jolted out of place). This would allow us to move this convolution filter to the center of the image then perform the deconvolution using any of the deconvolution method mentioned.

## Conclusion
We have introduced the process of blind deconvolution. The framework we have introduced is one that includes alternating minimization and optimizes for each variable separately in a loop. We have shown that these methods can be treated as black boxes, only thinking about the inputs and outputs.

## References

[1]:http://www2.math.uni-wuppertal.de/~klamroth/publications/gopfkl07.pdf
[1_doi]:https://dx.doi.org/10.1007%2Fs00186-007-0161-1
[2]:http://ieeexplore.ieee.org/xpls/abs_all.jsp?arnumber=5173518
[3]:http://www.osapublishing.org/josaa/fulltext.cfm?uri=josaa-12-1-58&id=33094
[4]:http://ieeexplore.ieee.org/xpls/abs_all.jsp?arnumber=661187&tag=1
[5]:http://ieeexplore.ieee.org/xpls/abs_all.jsp?arnumber=5995521

1. Gorski, Jochen; Pfeuffer, Frank; Klamroth, Kathrin (22 June 2007). ["Biconvex sets and optimization with biconvex functions: a survey and extensions"][1] (PDF). Mathematical Methods of Operations Research 66 (3): 373–407. [doi:10.1007/s00186-007-0161-1][1_doi]
2. Beck, Amir, and Marc Teboulle. "[Fast gradient-based algorithms for constrained total variation image denoising and deblurring problems.][2]" Image Processing, IEEE Transactions on 18.11 (2009): 2419-2434.
3. Fish, D. A., et al. "[Blind deconvolution by means of the Richardson–Lucy algorithm.][3]" JOSA A 12.1 (1995): 58-65.
4. Chan, Tony F., and Chiu-Kwong Wong. "[Total variation blind deconvolution.][4]" Image Processing, IEEE Transactions on 7.3 (1998): 370-375.
5. Krishnan, Dilip, Terence Tay, and Rob Fergus. "[Blind deconvolution using a normalized sparsity measure.][5]" Computer Vision and Pattern Recognition (CVPR), 2011 IEEE Conference on. IEEE, 2011.